In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from ultralytics import YOLO
import os
import boto3
import shutil

In [3]:
EPOCHS=int(os.environ.get("epochs", 50))
IMGSZ=640
NB_FROZEN_LAYER=10

DATA_CONFIG_PATH = 'utils/dataset.yaml'
LOCAL_DATASET_PATH = "dataset/"

BASE_MODEL_PATH = "models/yolo11x.pt"
TRAIN_MODEL_PATH = f"models/new-yolo11x{NB_FROZEN_LAYER}.pt"
TRAIN_MODEL_PATH_ONNX = f"models/new-yolo11x{NB_FROZEN_LAYER}.onnx"

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [4]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [5]:
client.download_file(AWS_S3_BUCKET, BASE_MODEL_PATH, "base-model.pt")

In [6]:
if os.path.exists('runs'):
    shutil.rmtree('runs')
model = YOLO('base-model.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ, freeze=NB_FROZEN_LAYER, batch=-1)

Ultralytics 8.3.146 🚀 Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA RTX A2000 12GB, 12032MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=utils/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=base-model.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

train: Scanning /opt/app-root/src/ai-demo/dataset/labels/train.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA RTX A2000 12GB) 11.75G total, 0.47G reserved, 0.47G allocated, 10.82G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    56876086       195.5         2.420         34.86         120.8        (1, 3, 640, 640)                    list
    56876086       390.9         3.014         52.73         88.34        (2, 3, 640, 640)                    list
    56876086       781.8         3.970         87.95         88.67        (4, 3, 640, 640)                    list
    56876086        1564         5.702         173.7         141.3        (8, 3, 640, 640)                    list
    56876086        3127         8.951         338.2           256       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 8 for CUDA:0 6.52G/11.75G (55%) ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5471.8±1267.3 MB/s, size: 908.7 KB)


train: Scanning /opt/app-root/src/ai-demo/dataset/labels/train.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]


...

      50/50       5.3G     0.5817      0.426     0.9063         15        640: 100%|██████████| 8/8 [00:02<00:00,  2.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.09it/s]

                   all         13         24      0.961      0.938      0.962      0.715



50 epochs completed in 0.104 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 114.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA RTX A2000 12GB, 12032MiB)
YOLO11x summary (fused): 190 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.17it/s]


                   all         13         24      0.957      0.944      0.961       0.73
                  Ball         11         11          1      0.964      0.995      0.807
                  Hoop         13         13      0.915      0.923      0.927      0.653
Speed: 0.1ms preprocess, 22.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train


In [7]:
model = YOLO('runs/detect/train/weights/best.pt')

In [8]:
model.export(format="onnx")

Ultralytics 8.3.146 🚀 Python-3.11.9 torch-2.6.0+cu126 CPU (Intel Core(TM) i7-14700F)
YOLO11x summary (fused): 190 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (109.1 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.53...
ONNX: export success ✅ 7.1s, saved as 'runs/detect/train/weights/best.onnx' (217.1 MB)

Export complete (12.2s)
Results saved to /opt/app-root/src/ai-demo/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=utils/dataset.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.onnx'

In [9]:
model_path = "runs/detect/train/weights/best.onnx"

In [10]:
client.upload_file(model_path, AWS_S3_BUCKET, TRAIN_MODEL_PATH_ONNX)

In [11]:
client.upload_file('runs/detect/train/weights/best.pt', AWS_S3_BUCKET, TRAIN_MODEL_PATH)

In [12]:
client.upload_file('runs/detect/train/results.csv', AWS_S3_BUCKET, 'results.csv')